# Pregunta 1

#### Nombre: Samuel Zúñiga
#### Número de alumno: 16637747

Importante: La mayoría del código expuesto en esta respuesta fue extraído de las clases de cátedra hechas por el profesor Martín Ugarte.

En primer lugar, tenemos las frecuencias de las letras en el lenguaje inglés.

In [ ]:
_DICT = {
    'A': .082, 'B': .015, 'C': .028, 'D': .043,  'E': .13,    'F': .022, 
    'G': .02,  'H': .061, 'I': .07,  'J': .0015, 'K': .0077,  'L': .04,   
    'M': .024, 'N': .067, 'O': .075, 'P': .019,  'Q': .00095, 'R': .06,   
    'S': .063, 'T': .091, 'U': .028, 'V': .0098, 'W': .024,   'X': .0015, 
    'Y': .02,  'Z': .00074
}

Funciones auxiliares y otras dadas en el enunciado.

In [ ]:
def char_probability(char, frequencies):
    return frequencies[char]

In [ ]:
def abs_distance(string, frequencies):
    return sum([
        abs(frequencies[c] - string.count(c) / len(string))
        for c in frequencies
    ])

La función `most_likely_char` recibe la posición, el texto cifrado, las frecuencias del idioma y el largo de la llave. Esta función retornará el carácter de la llave más probable para la posición indicada del mensaje del texto cifrado.

In [ ]:
def most_likely_char(pos, ciphertext, frequencies, key_length):
    """
    Toma el alfabeto
    """
    alphabet = [*frequencies.keys()]

    to_decrypt = [                                                              # todos los char del chiphertext que va a desencriptar un char especifico de la key
        ciphertext[i * key_length + pos]
        for i in range(len(ciphertext) // key_length)
    ]


    best_char = alphabet[0]                                                     # valor inicial para empezar a iterar en busca del best_char
    best_distance = len(alphabet)                                               # partimos con la peor distancia

    for candidate in alphabet:
        char_count = {c: 0 for c in alphabet}
        for c in to_decrypt:
            n_c = alphabet.index(c)
            n_candidate = alphabet.index(candidate)
            n = (n_c - n_candidate) % len(alphabet)                             # desencriptacion para OTP
            char_count[alphabet[n]] += 1

        diff = 0

        for c in alphabet:
            prob = char_count[c] / len(to_decrypt)                              # vemos la probabilidad de la letra en el texto
            diff += abs(char_probability(c, frequencies) - prob)                # calculamos su distancia para posteriormente buscar la menor
            
        if diff < best_distance:
            best_char = candidate
            best_distance = diff

    return best_char

Con la función anterior, iteramos sobre todo el largo de la llave para encontrar la key más probable.

In [ ]:
def prob_key(cipher, frequencies, key_length):

    to_ret =  ""
    to_ret = to_ret.join([
                 most_likely_char(i,cipher,frequencies, key_length)
                 for i in range(key_length)
                ])
    
    return to_ret

La siguiente función decripta un texto con una llave dada.

In [ ]:
def decrypt(cipher, prob_key, key_length, alphabet):

  plain_text=""
  for i in range(len(cipher)):
    n_c = alphabet.index(cipher[i])
    n_k = alphabet.index(
        prob_key[i % key_length]
    )
    plain_text += alphabet[(n_c - n_k) % len(alphabet)]
  
  return plain_text

La función `break_rp` recibe un texto cifrado, una frecuencia de carácteres y una función que indica cuánto dista un string de seguir una frecuencia de carácteres. En primer lugar iteraremos sobre todos los posibles largos para una llave dada, y guardaremos en un diccionario la distancia del texto decriptado a la frecuencia dada. Además haremos un diccionario con el texto y su llave. Finalmente retornaremos la llave para la cual se obtiene la mínima distancia.

In [ ]:
def break_rp(ciphertext,frequencies,distance):
    decripted_texts = dict()
    keys_from_text = dict()

    for i in range(1,len(ciphertext)//50):
        p_key = prob_key(ciphertext, frequencies, i)
        decripted_text = decrypt(ciphertext,p_key,i,list(frequencies.keys()))
        decripted_texts[decripted_text] = distance(decripted_text,frequencies)

        keys_from_text[decripted_text] = p_key

    return keys_from_text[min(decripted_texts, key=decripted_texts.get)]

Algunos tests:

In [ ]:
# def clean(t): return t.replace(" ", "").replace(",", "").replace("(", "").replace(")", "").replace(";", "").replace("'", "").replace(".", "").upper() # upper ya que en mi frecuencia test solo tengo mayúsculas

In [ ]:
# texto_cifrado = "cgxizjmjxsuicpubxjefnbhvquwjkqclvefhvmpqnbjeuyvcejknzkiumxrohhosfwkvtlywwlulxlljlxnkiynnvpztnzoaezltyimvtbtmveextkiumdjoimndbfeziubtikiyfzjtuzzsfcgbjfhmdeubbnkfwaizroxvgmubikfrmdjqubmveqboybltiupglztsymfvzuenfsyyzisywofbmtjefnbhvquwoyfhxvtivbofswavibwmzipzmcvqftdeuyqoztygxizjmzucsvjdccgdehcmrzubmcvdikmvtjhiujhzwzuikxybltxkflymfnnazgbxnnzoafjuvftmrexbozph"
# llave_utilizada = "UTVRB"

In [ ]:
# texto_cifrado = clean(texto_cifrado)

In [1]:
# break_rp(texto_cifrado, _DICT, abs_distance)